In [1]:
# Import libaries
import pandas as pd
import requests
import time

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


### Step 1: Scrape the korean pages soup for show titles

Note: Collect Title and show URL

In [ ]:
shows = []

for pg_num in range(1,36):
    KD_pg_url = 'https://www.viki.com/explore?country=korea&page=%s' %pg_num
    res = requests.get(KD_pg_url, headers={'User-agent': 'SB 1.0'})
    soup = BeautifulSoup(res.content, 'lxml')

    s_page = soup.find_all('a', {'class':'thumb-title strong'})
    for item in s_page:
        show = {}
        show['title'] = item.text.strip()
        show['url'] = 'https://www.viki.com'+ item.attrs['href']
        shows.append(show)

    time.sleep(1)
k_titles = pd.DataFrame(shows)

In [ ]:
k_titles.head()

In [ ]:
k_titles.to_csv('./data/k_titles.csv', index=False)

### Step 2: Using the show URL, scrape each show page and get user ratings

In [3]:
k_titles = pd.read_csv('./data/k_titles.csv')

In [96]:
show_url = k_titles['url'][1] + '#modal-review-see-all'

# use selenium webdriver as library that acts as a 
# headless browser to access modal window
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.get(show_url)
test = []

In [97]:
for i in range(711):
    time.sleep(5)
    soup_level1=BeautifulSoup(driver.page_source, 'lxml')

    table = soup_level1.find_all('ul',{'class':'media-list'})
    for row in table[2].find_all('li',{'class':'media'}):
        review = {}
        review['title'] = k_titles['title'][4]
        review['user'] = row.find('a').attrs['href'][7:]
        review['rating'] = row.find('span', {'class':'strong'}).text
        review['likes'] = (int(row.find_all('span', {'class':'count'})[0].text)
) + (int(row.find_all('span', {'class':'count'})[1].text)
)
        test.append(review)
    driver.find_element_by_partial_link_text('Next').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"partial link text","selector":"Next"}
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615355 (d5698f682d8b2742017df6c81e0bd8e6a3063189),platform=Mac OS X 10.13.6 x86_64)


In [98]:
testp = pd.DataFrame(test)
print(testp.shape[0])
print(testp['user'].nunique())
# reviews_5 = pd.DataFrame(reviews5)
# print(reviews_5.shape[0])
# print(reviews_5['user'].nunique())

516
516


In [104]:
testp

,likes,rating,title,user
0,91,10,My Strange Hero,snowglobeglitter
1,119,10,My Strange Hero,priszant_643
2,14,10,My Strange Hero,anna537
3,9,4,My Strange Hero,abena85
4,82,10,My Strange Hero,mezesjane_399
5,6,10,My Strange Hero,peppapig1239_82
6,33,10,My Strange Hero,jojohay
7,48,8,My Strange Hero,liobhan_514
8,20,10,My Strange Hero,kfriram_955
9,11,10,My Strange Hero,juracosta1410_358


In [41]:
%%html
<style>
table {float:left}
</style>

## scraped reviews
|movie|reported|retrieved|%|
|---|---|---|--|
|review 2 | 7106| 2195| 30.9|
|review 1 | 1354| 516| 38.1|
|review 3 | 3683| 1545| 41.9|
|review 4 | 4850| 1818| 37.5|
|review 5 | 2123| 660| 31.1|

In [71]:
660/2123

0.31088082901554404

In [95]:
#end the Selenium browser session
driver.quit()

In [73]:
reviews_5.to_csv('./data/show5_reviews.csv')

In [ ]:
reviews = []
for show in range(len(shows)):
    rest_url = k_titles['url']
    new_res = requests.get(rest_url)#modal-review-see-all
    soup = BeautifulSoup(new_res.content, 'lxml')

    user = soup.find_all('div', {'class':'media-body'})
    for item in user:
        review = {}
        review['user'] = item.find('a').attrs['href'][17:]
        show['rating'] = row.find('a').attrs['href'][12:].title()
        reviews.append(review)
# foods[100:150]

### Step 4: Create a pandas DataFrame from your list of foods

In [ ]:
df = pd.DataFrame(reviews)
df.head(10)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [ ]:
filepaths = ['./data/shows_part_1.csv', './data/shows_part_2.csv', './data/shows_part_3.csv','./data/shows_part_4.csv','./data/shows_part_5.csv']
df = pd.concat(map(pd.read_csv, filepaths))


In [ ]:
df

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.tail()

In [ ]:
df.to_csv('./data/pages_info.csv',index=False)